In [228]:
import os
from pathlib import Path
from bifacial_radiance import *
import pandas as pd
import shutil
import numpy as np

testfolder = str(Path().resolve().parent.parent /  'TEMP' /  'polysolar_dec')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\lm13\Documents\bifacial_radiance-main\TEMP\polysolar_dec


In [260]:
#FLUSH OBJECTS

dir = "objects"
objectsfolder = os.path.join(testfolder, dir)
os.makedirs(objectsfolder, exist_ok = True)
shutil.rmtree(objectsfolder) #deletes old objects in objects folder
os.makedirs(objectsfolder) #creates blank objects folder each time
print('Radiance objects have been flushed from %s' % objectsfolder)


Radiance objects have been flushed from C:\Users\lm13\Documents\bifacial_radiance-main\TEMP\polysolar_dec\objects


In [261]:
simulation_name = 'polysolar_correct_dec'
starttime='2023-12-1_0000'
endtime='2023-12-31_2300'
exp_name = 'plastic_gdl_checker_kjeller_5am'
#Location:
lat = 51.223 
lon = 0.317
# MakeModule Parameters
module ='test-module'
numpanels = 3  #number of modules per column of modules
x = 0.6
y = 1.2
xgap = 0.05 # Leaving 300 centimeters between modules on x direction
ygap = 0.052  # Leaving 5.2 centimeters between modules on y direction

sensorsy = 10
sensorsx = 10

# TorqueTube Parameters
axisofrotationTorqueTube = False
torqueTube = False
cellLevelModule = True
numcellsx = 100
numcellsy = 100
xcell = 0.002 #length of cell but in this case it is length of etched square 
ycell = 1.18 #height of etched square 
xcellgap = 0.0003 #etching gap
ycellgap = 0.002 #etching gap
glass = True
cellModule = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
              'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

# SceneDict Parameters
pitch = 15 # m (distance between rows, cannot be zero, doesn't matter if there is only 1 row)
albedo = 0.2 # ground albedo
clearance_height = 0.3 #30cm 

nMods = 14 #14 # 7 number of columns
nRows = 1 # only 1 row, mounted on the south side of the greenhouse
azimuth_ang = 200 
tilt = 90 #35 # tilt. 
cumulativesky = False

In [262]:
with open('C:/Users/lm13/Documents/bifacial_radiance-main/TEMP/polysolar_dec/materials/ground.rad') as f:
 materialslist = f.readlines()
if any("#greenhouse plastic" in s for s in materialslist):
 print("Material found")
 f.close()
else:
 print("Appending")
 materialslist.append('\n\n#greenhouse plastic white\n') #material description
 materialslist.append('void trans greenhouse_plastic\n') #material properties and call according to Radiancedocumentation
 materialslist.append('0\n')
 materialslist.append('0\n')
 materialslist.append('7 1 1 1 \n')
 materialslist.append('0.05 0.02 \n') #'0.04 0.05 \n') #this material was sourced from similar study in Radiance https://www-sciencedirect-com.ezproxy.uio.no/science/article/pii/S0306261921016986
 materialslist.append('0.7 0.3') #'0.869 0.22') # Translucent cover - SG80 - 84% hemispherical tranmittance, 78% haze factor
 materialslist.append('1.231 0.930 0.778')
 updateRad = open("materials/ground.rad", "w")
 print("Writing to new file")
 for m in materialslist:
     updateRad.write(m)
 updateRad.close()
 f.close()

Material found


In [264]:
apv = RadianceObj(simulation_name, path = testfolder) # Create a RadianceObj 'object' with simulation name
apv.setGround(0.2)


path = r'C:\Users\lm13\Documents\bifacial_radiance-main\bifacial_radiance\TEMP'

epwfile = epwfile = apv.getEPW(51.223, 0.317) #read weather file
metdata = apv.readWeatherFile(epwfile,'2019-05-20_0000', '2019-05-20_2300', coerce_year=2019)
#timestamp = metdata.datetime.index(pd.to_datetime('2019-06-17 12:0:0 +0')) 
#apv.gendaylit(timestamp)  # Mid-day, June 17th

#epwfile = r'C:\Users\lm13\Documents\bifacial_radiance-main\solcast_jan-june_correct.epw'
#metdata = apv.readWeatherFile(epwfile,'2013-05-20_0000', '2013-05-20_2300', coerce_year=2013 )

GHI = metdata.ghi
Datetime = metdata.datetime
epw_len = len(GHI)
timestamps = np.array(list(range(1, epw_len)))



path = C:\Users\lm13\Documents\bifacial_radiance-main\TEMP\polysolar_dec
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: GBR_London.Gatwick.037760_IWEC.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2019
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [240]:
#Create new material for caravan:
description = 'Pale Aqua Green'
ref_material_name = 'cement'

Rrefl = 0.844 
Grefl = 0.867
Brefl = 0.727 
apv.addMaterial(material=ref_material_name, Rrefl=Rrefl, Grefl=Grefl, Brefl=Brefl, comment=description)

Material exists, overwriting...

Added material cement to file ground.rad


In [249]:
moduleDict = apv.makeModule(name=module,x=x,y=y,numpanels = numpanels, xgap=xgap, ygap=ygap)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch':pitch,'clearance_height':clearance_height,'azimuth':azimuth_ang, 'module_type':module, 'nMods': nMods, 'nRows': nRows, 'originx': (-1-5.638), 'originy': (-31.80922 +2.2)}

scene = apv.makeScene(module, sceneDict, '!xform -rz -20') #makeScene creates a .rad file with modules
#!xform -rz -20 rotates it by 20 degrees about the z axis to make it south west orientation rather than just south 
octfile = apv.makeOct(apv.getfilelist()) # makeOct combines all of the ground, sky and object fil|es into a .oct file.



# there is no torquetube this just represents the length of the south facing greenhouse wall
torquetubelength = 40
# torquetube 1
torquetubelength = 40
# torquetube 1




octfile = apv.makeOct(apv.getfilelist()) # makeOct combines all of the ground, sky and object files into a .oct file



Module Name: test-module
Module test-module updated in module.json
Created polysolar_correct_dec.oct
Created polysolar_correct_dec.oct


In [250]:
!rvu -vf views\front.vp -e errors.txt -pe 0.5 -vp -34 -30 10 -vd 0.7 0.7 -0.15  polysolar_correct_dec.oct

In [265]:
for ts in timestamps:
    print("timestamp:"+str(ts))
    ts_string = str(ts).zfill(4)
    ghi_ts = GHI[ts]    
    
    simulation_string = simulation_name+'_w'+'_ts'+ts_string
    apv = RadianceObj(simulation_string,str(testfolder))
    metdata = apv.readWeatherFile(epwfile)
    apv.setGround(metdata.albedo[ts])
    
    if ghi_ts == 0:
        print("Skip timestamp "+ts_string+", GHI = 0")
    else:
        apv.gendaylit(ts) #produces sky
        apv.setGround(metdata.albedo[ts])
        moduleDict = apv.makeModule(name=module,x=x,y=y,numpanels = numpanels, xgap=xgap, ygap=ygap)
        # create a scene with all the variables
        sceneDict = {'tilt':tilt,'pitch':pitch,'clearance_height':clearance_height,'azimuth':azimuth_ang, 'module_type':module, 'nMods': nMods, 'nRows': nRows, 'originx': (-1-5.638), 'originy': (-31.80922 +2.2)}

        scene = apv.makeScene(module, sceneDict, '!xform -rz -20') #makeScene creates a .rad file with modules
        #!xform -rz -20 rotates it by 20 degrees about the z axis to make it south west orientation rather than just south 
        octfile = apv.makeOct(apv.getfilelist()) # makeOct combines all of the ground, sky and object fil|es into a .oct file.


        # there is no torquetube this just represents the length of the south facing greenhouse wall
        torquetubelength = 40
        # torquetube 1
        torquetubelength = 40
        # torquetube 1
        '''

        name='GlassGreenhouseWallSouthupper'
        text='! genbox greenhouse_plastic greenhousewall_Su {} 0.1 0.2 | xform -t {} -0.1 0 | xform -t 0 -30.1 4.01'.format(torquetubelength , (-torquetubelength)/2.0) # 28 m wall, 4.2 meters high
        customObject = apv.makeCustomObject(name,text)
        apv.appendtoScene(radfile=scene.radfiles, customObject=customObject, text='!xform -rz -20')

        name='GlassGreenhouseWallSouthlower'
        text='! genbox greenhouse_plastic greenhousewall_Su {} 0.3 0.3 | xform -t {} -0.1 0 | xform -t 0 -30.1 0'.format(torquetubelength , (-torquetubelength)/2.0) # 28 m wall, 4.2 meters high
        customObject = apv.makeCustomObject(name,text)
        apv.appendtoScene(radfile=scene.radfiles, customObject=customObject, text='!xform -rz -20')

        name='GlassGreenhouseWallNorth'
        text='! genbox greenhouse_plastic greenhousewall_N {} 0.001 4.2 | xform -t {} -0.1 0 | xform -t 0 30 0'.format(torquetubelength, (-torquetubelength)/2.0) # 28 m wall, 4.2 meters high
        customObject = apv.makeCustomObject(name,text)
        apv.appendtoScene(radfile=scene.radfiles, customObject=customObject, text='!xform -rz -20')


        name='Greenhouse_Floor_small'
        text='! genbox white_EPDM CenterPatch_s {} 60 0.01 | xform -t {} -30 0'.format(torquetubelength,(-torquetubelength)/2.0)
        customObject = apv.makeCustomObject(name,text)
        apv.appendtoScene(scene.radfiles, customObject, '!xform -rz -20')

        name='caravan'
        text='! genbox cement caravan {} 5 4.2 | xform -t {} -0.1 0 | xform -t 24.1 -40 0'.format(15 , (-torquetubelength)/2.0 - 10)
        customObject = apv.makeCustomObject(name,text)
        apv.appendtoScene(radfile=scene.radfiles, customObject=customObject, text='!xform -rz -20')

        '''



        octfile = apv.makeOct(apv.getfilelist()) # makeOct combines all of the ground, sky and object files into a .oct file
        
        analysis = AnalysisObj(octfile, apv.name) # return an analysis object including the scan dimensions for back irradiance
        frontscan, backscan = analysis.moduleAnalysis(scene)
        analysis.analysis(octfile, simulation_string, frontscan, backscan) #str (starttime)+"_to_"+str(endtime), frontscan, backscan) #, accuracy='high')


timestamp:1
path = C:\Users\lm13\Documents\bifacial_radiance-main\TEMP\polysolar_dec
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.000 avg
0 nonzero albedo values.
Loading albedo, 1 value(s), 0.000 avg
0 nonzero albedo values.

Module Name: test-module
Module test-module updated in module.json
Created polysolar_correct_dec_w_ts0001.oct
Created polysolar_correct_dec_w_ts0001.oct
Linescan in process: polysolar_correct_dec_w_ts0001_Front
Linescan in process: polysolar_correct_dec_w_ts0001_Back
Saved: results\irr_polysolar_correct_dec_w_ts0001.csv
timestamp:2
path = C:\Users\lm13\Documents\bifacial_radiance-main\TEMP\polysolar_dec
8760 line in WeatherFile. Assuming

Created polysolar_correct_dec_w_ts0009.oct
Created polysolar_correct_dec_w_ts0009.oct
Linescan in process: polysolar_correct_dec_w_ts0009_Front
Linescan in process: polysolar_correct_dec_w_ts0009_Back
Saved: results\irr_polysolar_correct_dec_w_ts0009.csv
timestamp:10
path = C:\Users\lm13\Documents\bifacial_radiance-main\TEMP\polysolar_dec
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.000 avg
0 nonzero albedo values.
Loading albedo, 1 value(s), 0.000 avg
0 nonzero albedo values.

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

Created polysolar_correct_dec_w_ts0010.oct
Created

In [266]:
resultsfolder = os.path.join(testfolder, 'results')

columns = ['Datetime','GHI',
           'avg_FrontIrrad','avg_RearIrrad','avg_TotalIrrad']

results_df = pd.DataFrame(columns=columns,index=timestamps)
avg_FrontIrrad = []
avg_RearIrrad = []


for ts in timestamps:

    ghi_ts = GHI[ts] 
    results_df.loc[ts,'GHI']=ghi_ts
    results_df.loc[ts,'Datetime']=Datetime[ts]
    
    if ghi_ts != 0:
        
        ts_string = str(ts).zfill(4) 
        simulation_string = simulation_name+'_w'+'_ts'+ts_string
        sim_name = ('irr_'+simulation_string+'.csv')
        result_file = os.path.join(resultsfolder, sim_name)
        data = pd.read_csv(result_file)  
        
        FrontIrrad = data['Wm2Front'].to_numpy(dtype='float')
        RearIrrad = data['Wm2Back'].to_numpy(dtype='float')
        results_df.loc[ts,'avg_FrontIrrad']=FrontIrrad.mean()
        results_df.loc[ts,'avg_RearIrrad']=RearIrrad.mean()
results_df['Datetime'] = pd.to_datetime(results_df['Datetime'])
results_df['Datetime'] = results_df['Datetime'].dt.tz_localize(None)

In [267]:
results_df.to_csv('RESULTS.csv')

^C


Linescan in process: polysolar_correct_dec_modules_plastic_gdl_checker_kjeller_5am_Front
Linescan in process: polysolar_correct_dec_modules_plastic_gdl_checker_kjeller_5am_Back
Saved: results\irr_polysolar_correct_dec_modules_plastic_gdl_checker_kjeller_5am.csv


FileNotFoundError: [Errno 2] No such file or directory: 'results\\irr_polysolar_modules_plastic_gdl_checker_kjeller_5am.csv'